In [22]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

In [33]:
input_file = "/Users/lilblick/Downloads/kyverna_study_data.xlsx"
full_output = "/Users/lilblick/Downloads/kyverna_study_full.xlsx"
summary_output = "/Users/lilblick/Downloads/kyverna_study_summary.xlsx"

times_of_interest = ["DAY28", "DAY42", "DAY56",
               "MON3", "MON4", "MON5", "MON6",
               "MON9", "MON12", "MON18"]

grade_thresholds = {
    "WBC (x10^9/L)": {"Grade3": 2.0, "Grade4": 1.0},
    "Abs. Neutrophils (x10^9/L)": {"Grade3": 1.0, "Grade4": 0.5},
    "Abs. Lymphocytes (x10^9/L)": {"Grade3": 0.5, "Grade4": 0.20}
}

# Color coding
grade3_fill = PatternFill(start_color="0000FF", end_color="0000FF", fill_type="solid") # blue for Grade3
grade4_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid") # Red for Grade4

In [34]:
# Load data first
df = pd.read_excel(input_file, sheet_name="in")
print("Loaded data shape:", df.shape)
print(df.head())

Loaded data shape: (654, 10)
  UniqueSubjectIdentifier  Site.ID      Cohort                   Parameter  \
0              840002-001   840002  ABC101-001  Abs. Lymphocytes (x10^9/L)   
1              840002-001   840002  ABC101-001  Abs. Neutrophils (x10^9/L)   
2              840002-001   840002  ABC101-001               WBC (x10^9/L)   
3              840002-001   840002  ABC101-001  Abs. Lymphocytes (x10^9/L)   
4              840002-001   840002  ABC101-001  Abs. Neutrophils (x10^9/L)   

  Analysis.Visit  Analysis.Value  Analysis.RelativeDay       Date Dosing.Date  \
0      Screening            1.19                   -50 2023-08-07  2023-09-26   
1      Screening            5.62                   -50 2023-08-07  2023-09-26   
2      Screening            7.21                   -50 2023-08-07  2023-09-26   
3            Aph            0.67                   -36 2023-08-21  2023-09-26   
4            Aph            4.92                   -36 2023-08-21  2023-09-26   

              C

In [35]:
# Inspect visit names
print("\nUnique Analysis.Visit values BEFORE filtering:")
print(df["Analysis.Visit"].unique())


Unique Analysis.Visit values BEFORE filtering:
['Screening' 'Aph' 'LD' 'UNS' 'DAY0' 'DAY1' 'DAY3' 'DAY7' 'DAY10' 'DAY14'
 'DAY17' 'DAY21' 'DAY28' 'DAY42' 'DAY56' 'MON3' 'MON4' 'MON5' 'MON6'
 'MON9' 'MON12' 'MON18' 'DAY5']


In [36]:
# Filter only relevant timepoints
df = df[df["Analysis.Visit"].isin(times_of_interest)]
print("\nAfter filtering by timepoints:", df.shape)
print(df["Analysis.Visit"].value_counts())


After filtering by timepoints: (240, 10)
Analysis.Visit
DAY28    30
DAY42    30
MON3     30
MON4     30
MON5     30
DAY56    24
MON6     24
MON9     18
MON12    15
MON18     9
Name: count, dtype: int64


In [37]:
def grade_label(param, value):
    """
    This function adds grade labels to cells that cross the established thresholds

    :param: the parameter of interest. i.e Abs. Lymphocytes (x10^9/L)
    :value: the value thresholds for said parameter. i.e 
            "Abs. Lymphocytes (x10^9/L)": {"Grade3": 2.0, "Grade4": 1.0},
    """
    if param not in grade_thresholds:
        return ""
    t = grade_thresholds[param]
    if value < t["Grade4"]:
        return "Grade4"
    elif value < t["Grade3"]:
        return "Grade3"
    else:
        return ""

df["Grade"] = df.apply(lambda x: grade_label(x["Parameter"], x["Analysis.Value"]), axis=1)

In [38]:
# Annotated string for summary
df["Annotated"] = df.apply(
    lambda x: f"{x['Analysis.Value']}({x['Grade']})" if x["Grade"] else str(x["Analysis.Value"]),
    axis=1
)
print("\nSample with grades + annotations:")
print(df.head(10))


Sample with grades + annotations:
   UniqueSubjectIdentifier  Site.ID      Cohort                   Parameter  \
36              840002-001   840002  ABC101-001  Abs. Lymphocytes (x10^9/L)   
37              840002-001   840002  ABC101-001  Abs. Neutrophils (x10^9/L)   
38              840002-001   840002  ABC101-001               WBC (x10^9/L)   
39              840002-001   840002  ABC101-001  Abs. Lymphocytes (x10^9/L)   
40              840002-001   840002  ABC101-001  Abs. Neutrophils (x10^9/L)   
41              840002-001   840002  ABC101-001               WBC (x10^9/L)   
42              840002-001   840002  ABC101-001  Abs. Lymphocytes (x10^9/L)   
43              840002-001   840002  ABC101-001  Abs. Neutrophils (x10^9/L)   
44              840002-001   840002  ABC101-001               WBC (x10^9/L)   
45              840002-001   840002  ABC101-001  Abs. Lymphocytes (x10^9/L)   

   Analysis.Visit  Analysis.Value  Analysis.RelativeDay       Date  \
36          DAY28        

In [39]:
df.to_excel(full_output, index=False)

# Save subject-level pivot summaries
with pd.ExcelWriter(summary_output, engine="openpyxl") as writer:
    for subj, g in df.groupby("UniqueSubjectIdentifier"):
        pivot = g.pivot_table(
            index=["Parameter"],
            columns="Analysis.Visit",
            values="Annotated",
            aggfunc="first"
        ).reset_index()
        
        # Each subject gets its own sheet
        pivot.to_excel(writer, sheet_name=str(subj), index=False)

print("\n✅ Subject-level summary saved (1 sheet per subject):", summary_output)


✅ Subject-level summary saved (1 sheet per subject): /Users/lilblick/Downloads/kyverna_study_summary.xlsx


In [40]:
wb = load_workbook(full_output)
ws = wb.active

# Identify column indexes dynamically
headers = {cell.value: idx for idx, cell in enumerate(next(ws.iter_rows(min_row=1, max_row=1)), start=1)}
value_col = headers.get("Analysis.Value")
grade_col = headers.get("Grade")

print("\nApplying formatting to full dataset...")
for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    grade = row[grade_col - 1].value
    if grade == "Grade3":
        row[value_col - 1].fill = grade3_fill
    elif grade == "Grade4":
        row[value_col - 1].fill = grade4_fill

wb.save(full_output)
print("✅ Coloring applied to full dataset.")


Applying formatting to full dataset...
✅ Coloring applied to full dataset.


In [41]:
wb = load_workbook(summary_output)

# loop over each subject sheet
for ws in wb.worksheets:  
    for row in ws.iter_rows(min_row=2, min_col=2):
        for cell in row:
            if cell.value and isinstance(cell.value, str):
                if "(Grade3)" in cell.value:
                    cell.fill = grade3_fill
                elif "(Grade4)" in cell.value:
                    cell.fill = grade4_fill

wb.save(summary_output)
print("✅ Coloring applied to summary workbook.")

print("\n🎉 Finished. Files saved:")
print("   Full data:", full_output)
print("   Summary  :", summary_output)

✅ Coloring applied to summary workbook.

🎉 Finished. Files saved:
   Full data: /Users/lilblick/Downloads/kyverna_study_full.xlsx
   Summary  : /Users/lilblick/Downloads/kyverna_study_summary.xlsx
